In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/AhmedSalahBasha/SchemaMatching.git

In [ ]:
!git clone https://github.com/davidfoerster/schema-matching.git

In [ ]:
!./content/drive/MyDrive/Colab Notebooks/schema-matching/demo/1-match.sh

In [ ]:
!pip install selenium

In [ ]:
!pip install webdriver-manager

In [ ]:
import requests
from lxml import html
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

#driver.get("https://www.webite-url.com")

In [ ]:
class HtmlGetter:
   def get_html(self, url):
       pass

class HtmlParseGetter(HtmlGetter):
   def __init__(self, subject):
       self.subject = subject
   def get_html(self, url):
       html_source = self.subject.get_html(url)
       html_element = html.fromstring(html_source)
       return html_element

class SeleniumHtmlGetter(HtmlGetter):
   def __init__(self, scroll_to_bottom=False):
       self.scroll_to_bottom = scroll_to_bottom
   def get_html(self, url):
      chrome_options = webdriver.ChromeOptions()
      chrome_options.add_argument('--headless')
      chrome_options.add_argument('--no-sandbox')
      chrome_options.add_argument('--disable-dev-shm-usage')
      browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
      browser.maximize_window()
      browser.get(url)
      html_source = browser.page_source
      browser.quit()
      return html_source

if __name__ == '__main__':
   url = 'https://tiki.vn/dien-thoai-smartphone/c1795?page=2'
   html_getter = HtmlParseGetter(SeleniumHtmlGetter( ))
   html_tree = html_getter.get_html(url)
   for v in html_tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "product-item", " " ))]'):
     print(v.xpath('//a/@href'))
     break


In [ ]:
!pip install requests

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
import sys
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

with open("tiki_crawl.txt", mode='w', encoding='utf8') as f:
  for index in range(1,10):
    response = requests.get(f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=48&include=advertisement&aggregations=2&trackity_id=d743e974-1a06-9329-8e89-e909bdbed2ae&category=1795&page={index}&urlKey=dien-thoai-smartphone", headers = headers)
    datas_json = json.loads(response.text)['data']
    for idx in range(len(datas_json)):
      productId = datas_json[idx]['id']
      response_detail = requests.get(f"https://tiki.vn/api/v2/products/{productId}", headers = headers)

      phone_detail = dict()
      phone_detail["Tên sản phẩm"] = response_detail.json()['name']
      phone_detail["Giá"] = response_detail.json()['price']
      for features in response_detail.json()['specifications']:
        for feature in features['attributes']:
          value = BeautifulSoup(feature['value'], "lxml").text.strip()
          phone_detail[feature['name']] = value
      f.write(json.dumps(phone_detail, ensure_ascii=False) + "\n")

In [ ]:
columns = set() 
with open("/content/tiki_crawl.txt", mode = 'r', encoding = 'utf8') as g:
  for idx, line in enumerate(g):
    line = json.loads(line)
    columns.update(set(line.keys()))

In [ ]:
import pandas as pd
df = pd.DataFrame(columns = columns)

In [ ]:
df

In [ ]:
with open("/content/tiki_crawl.txt", mode = 'r', encoding = 'utf8') as g:
  for idx, line in enumerate(g):
    line = json.loads(line)
    df = df.append(line, ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.to_csv("tiki_crawl.csv", sep='\t', encoding='utf-8')

In [ ]:
fd = pd.read_csv("/content/tiki_crawl.csv", sep='\t', encoding = 'utf8', index_col=0)

In [ ]:
fd